In [166]:
import geopandas as gpd
import pandas as pd
import folium as f
ca = gpd.read_file("data_sources/community_areas.geojson")
male_chl = pd.read_csv("data_sources/chlamydia_males.csv")
male_gon = pd.read_csv("data_sources/gonorrhea_males.csv")
female_chl = pd.read_csv("data_sources/chlamydia_females.csv")
female_gon = pd.read_csv("data_sources/gonorrhea_females.csv")

centers = pd.read_csv("data_sources/centers_clean.csv")
centers['Longitude'] = centers['Longitude'].astype(float)

centers.dtypes


Venue Type          object
Name                object
Address             object
City                object
State               object
ZIP Code             int64
Full Address        object
Latitude           float64
Longitude          float64
Community Areas     object
dtype: object

In [167]:
male_chl = male_chl[["Community Area Name",
                     "Incidence Rate 2000",
                     "Incidence Rate 2001",
                     "Incidence Rate 2002",
                     "Incidence Rate 2003",
                     "Incidence Rate 2004",
                     "Incidence Rate 2005",
                     "Incidence Rate 2006",
                     "Incidence Rate 2007",
                     "Incidence Rate 2008",
                     "Incidence Rate 2009",
                     "Incidence Rate 2010",
                     "Incidence Rate 2011",
                     "Incidence Rate 2012",
                     "Incidence Rate 2013",
                     "Incidence Rate 2014"]]



male_gon = male_gon[["Community Area Name",
                     "Incidence Rate 2000",
                     "Incidence Rate 2001",
                     "Incidence Rate 2002",
                     "Incidence Rate 2003",
                     "Incidence Rate 2004",
                     "Incidence Rate 2005",
                     "Incidence Rate 2006",
                     "Incidence Rate 2007",
                     "Incidence Rate 2008",
                     "Incidence Rate 2009",
                     "Incidence Rate 2010",
                     "Incidence Rate 2011",
                     "Incidence Rate 2012",
                     "Incidence Rate 2013",
                     "Incidence Rate 2014"]]


male_gon_long = pd.melt(male_gon,
                        id_vars = ['Community Area Name'],
                        var_name = "Temp",
                        value_name = "Incidences")

male_chl_long = pd.melt(male_chl,
                        id_vars = ['Community Area Name'],
                        var_name = "Temp",
                        value_name = "Incidences")


male_gon_long['STI Name'] = 'GON'
male_chl_long['STI Name'] = 'CHL'
male_gon_long['Gender'] = 'MALE'
male_chl_long['Gender'] = 'MALE'
male_chl_long['Year'] = male_chl_long['Temp'].str.extract(r'([0-9]{4})', expand = True)
male_gon_long['Year'] = male_gon_long['Temp'].str.extract(r'([0-9]{4})', expand = True)
male_sti = [male_gon_long, male_chl_long]

male_sti = pd.concat(male_sti)
male_sti = male_sti[['Community Area Name',
                   'Incidences',
                   'STI Name',
                   'Gender',
                   'Year']]
male_sti = male_sti.fillna(0)
male_sti = male_sti.replace("<5", 0)

In [168]:
female_chl = female_chl[["Community Area Name",
                     "Incidence Rate 2000",
                     "Incidence Rate 2001",
                     "Incidence Rate 2002",
                     "Incidence Rate 2003",
                     "Incidence Rate 2004",
                     "Incidence Rate 2005",
                     "Incidence Rate 2006",
                     "Incidence Rate 2007",
                     "Incidence Rate 2008",
                     "Incidence Rate 2009",
                     "Incidence Rate 2010",
                     "Incidence Rate 2011",
                     "Incidence Rate 2012",
                     "Incidence Rate 2013",
                     "Incidence Rate 2014"]]



female_gon = female_gon[["Community Area Name",
                     "Incidence Rate 2000",
                     "Incidence Rate 2001",
                     "Incidence Rate 2002",
                     "Incidence Rate 2003",
                     "Incidence Rate 2004",
                     "Incidence Rate 2005",
                     "Incidence Rate 2006",
                     "Incidence Rate 2007",
                     "Incidence Rate 2008",
                     "Incidence Rate 2009",
                     "Incidence Rate 2010",
                     "Incidence Rate 2011",
                     "Incidence Rate 2012",
                     "Incidence Rate 2013",
                     "Incidence Rate 2014"]]


female_gon_long = pd.melt(female_gon,
                        id_vars = ['Community Area Name'],
                        var_name = "Temp",
                        value_name = "Incidences")

female_chl_long = pd.melt(female_chl,
                        id_vars = ['Community Area Name'],
                        var_name = "Temp",
                        value_name = "Incidences")


female_gon_long['STI Name'] = 'GON'
female_chl_long['STI Name'] = 'CHL'
female_gon_long['Gender'] = 'FEMALE'
female_chl_long['Gender'] = 'FEMALE'
female_chl_long['Year'] = female_chl_long['Temp'].str.extract(r'([0-9]{4})', expand = True)
female_gon_long['Year'] = female_gon_long['Temp'].str.extract(r'([0-9]{4})', expand = True)
female_sti = [female_gon_long, female_chl_long]

female_sti = pd.concat(female_sti)
female_sti = female_sti[['Community Area Name',
                   'Incidences',
                   'STI Name',
                   'Gender',
                   'Year']]
female_sti = female_sti.fillna(0)
female_sti = female_sti.replace("<5", 0)



In [169]:
incidence_sti = [male_sti, female_sti]
incidence_sti = pd.concat(incidence_sti)
incidence_sti = incidence_sti[(incidence_sti["Community Area Name"]!= "Chicago") &(incidence_sti["Community Area Name"]!= "Unknown")]
incidence_sti['Community Area Name'] = incidence_sti['Community Area Name'].str.upper()

incidence_sti.to_csv("data_sources/incidences_sti.csv")

In [170]:
#mask incidence to show only 2014 values
incidence_sti_2014 = incidence_sti[incidence_sti['Year'] == '2014']

#group by to get total incidence rate

incidence_sti_2014 = incidence_sti_2014.groupby("Community Area Name").sum().reset_index()

In [171]:
x = ca.merge(incidence_sti_2014, left_on = 'community',
            right_on = "Community Area Name",
            how = 'inner')

In [172]:
m = f.Map(location = [41.881832, -87.623177],
              tiles = 'cartodbpositron',
              zoom_start = 10)


ft = "Number of Reported Cases of STI"

colormap = f.LinearColormap(("Green", "Yellow","Orange", "Red"), 
                                 vmin = min(x['Incidences']), 
                                 vmax = max(x['Incidences']) *.5,
                                 caption = "Number of Incidences")


colormap.add_to(m)

f.GeoJson(x,
               style_function = lambda feature: { 
                  'fillColor': colormap(feature['properties']["Incidences"]),
                   "color" : "k", 
                   "weight" : 0.3, 
                   "fillOpacity" : 0.5,
                   "popup": feature['properties']['community']
               },
         name = "Community Areas").add_to(m)

for l in range(0, len(centers)-1):
    lat = centers['Latitude'][l]
    lng = centers['Longitude'][l]
    name = centers['Name'][l]
    
    f.Marker([lat, lng],
             popup = name).add_to(m)


f.LayerControl().add_to(m)
m.save("incidence_map.html")
